<div style='float: right'><img src='pic/block.png'/></div>
## <div id='block' />ブロックパズル

In [ ]:
import numpy as np
from itertools import product
from pulp import *
from ortoolpy import addvar, addvars, addbinvar, addbinvars
from unionfind import unionfind
data = """\
AAECD
CDCDE
ABDBA
DEAEC
ECBBB""".split()
nw, nh = len(data[0]), len(data)
nb = len(set(list(''.join(data)))) # 1ブロックの大きさ
ns = nw * nh // nb # ブロック数

### 問題
* 部品と部品の間に線を引いて、盤面を指定されたブロックに分けます
* ブロックの中に部品がちょうど1つ入るようにします
* それぞれのブロックに入る部品は、どれもタテかヨコにつながっているように

### 変数
* v：ブロック候補を選ぶかどうか # (1)

### 制約
* 各マスで選ばれた候補はちょうど1つ # (2)

In [ ]:
chk = [False] * ns
cand = [] # 候補
def makecand(c, ca):
    if len(ca) == ns:
        if unionfind.isconnectedlist(nw, nh, ca):
            cand.append(ca)
        return
    if c == nw * nh:
        return
    i, j = c % nw, c // nw
    k = ord(data[i][j]) - 65
    if not chk[k]:
        chk[k] = True
        makecand(c + 1, ca + [(i, j)])
        chk[k] = False
    makecand(c + 1, ca)
makecand(0, [])
nn = len(cand)

In [ ]:
m = LpProblem()
v = addbinvars(nn) # (1)
e = [[LpAffineExpression() == 1 for j in range(nw)] for i in range(nh)]
for i, j in product(range(nh), range(nw)):
    m += e[i][j] # (2)
for k in range(nn):
    for i, j in cand[k]:
        e[i][j].addterm(v[k], 1)
%time m.solve()
r = [[0]*nw for _ in range(nh)]
for i, j in enumerate(np.array(cand)[np.vectorize(value)(v) == 1]):
    for y, x in j:
        r[y][x] = i
r